In [ ]:
import pandas as pd

# Load the data
data_path = '/path_to_your_data/public_data.csv'
data = pd.read_csv(data_path)
naics_data_path = '/path_to_your_data/aggregated_firm_sizes.xlsx'
naics_data = pd.read_excel(naics_data_path)

# Clean the state names in both datasets
data['victim.state'] = data['victim.state'].str.upper().replace(r'^US-', '', regex=True)
naics_data['StateName'] = naics_data['StateName'].replace({
    'Alabama': 'AL', 'Alaska': 'AK', ...  # Full mapping of states
})

# Calculate the total incidents and total firms
total_incidents = data['victim.state'].value_counts().sum()
total_firms = naics_data['Firms'].sum()

# Prepare the Risk Factors DataFrame
risk_factors = pd.DataFrame(data['victim.state'].value_counts()).reset_index()
risk_factors.columns = ['State', 'Number of Incidents']
risk_factors = risk_factors.merge(naics_data, left_on='State', right_on='StateName', how='left')

# Calculate Risk Factor with a Weighting Factor of 1
risk_factors['Risk Factor'] = (risk_factors['Number of Incidents'] / total_incidents) * \
                              (risk_factors['Firms'] / total_firms) * 1  # Assuming weighting factor as 1 for all

# Normalize the Risk Factor to be between 1 and 100
max_risk_factor = risk_factors['Risk Factor'].max()
risk_factors['Normalized Risk Factor'] = (risk_factors['Risk Factor'] / max_risk_factor) * 100
risk_factors['Normalized Risk Factor'] = risk_factors['Normalized Risk Factor'].clip(lower=1)

# Save the DataFrame to an Excel file
output_excel_path = '/path_to_your_data/public_normalized_risks.xlsx'
risk_factors.to_excel(output_excel_path, index=False)
